In [23]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim


### Read datafile

In [29]:
filePath =  "../../project_course_data/"
fileName = "preprocessedDataLabeled.csv"

data = pd.read_csv(filePath + fileName)

C:\Users\linus\AppData\Local\Temp\ipykernel_11360\1126002880.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filePath + fileName)


### Function for debugging

In [30]:
def printRow(df, id):
    try:
        row = df[df["Id"] == id]
        for col, val in row.iloc[0].items():
            print(f"{col}: {val}")
    except KeyError:
        print(f"Row with id {id} not found in DataFrame.")

In [31]:
id = 1
printRow(data, id)

Datetime: 2023-11-22 12:17:02
Label: Unknown
Id: 1
Duration: 135.108
Packets: 100
Bytes: 4000
Flows: 1
Src IP Addr: 3.67.35.217
Src Port: 443
Dst IP Addr: 192.168.8.235
Dst Port: 63931
Host IP: 3.67.35.217
Client IP: 192.168.8.235
Domain Name: ec2-3-67-35-217.eu-central-1.compute.amazonaws.com
IGMP: 0
TCP: 1
UDP: 0
Host Port: 443
0: 0
1900: 0
2099: 0
22222: 0
27017: 0
27018: 0
27022: 0
27023: 0
27024: 0
27025: 0
27028: 0
27029: 0
27032: 0
27035: 0
27036: 0
27037: 0
27043: 0
27047: 0
27051: 0
27052: 0
27053: 0
27057: 0
27060: 0
3478: 0
3480: 0
34820: 0
3702: 0
4070: 0
44142: 0
443: 1
50002: 0
50012: 0
50022: 0
50027: 0
5129: 0
5223: 0
5228: 0
5353: 0
5355: 0
67: 0
80: 0
8009: 0
1drv: 0
1.00E+100: 0
82f3dc: 0
833aec: 0
a104: 0
a184: 0
a2: 0
a23: 0
a95: 0
aa784e235de7c8b14: 0
adobedc: 0
akamaitechnologies: 0
all: 0
amazonaws: 1
ams: 0
ams1: 0
ams15s47: 0
ams15s51: 0
ams17s02: 0
ams17s04: 0
ams17s13: 0
ams17s17: 0
ams58: 0
andreas: 0
arn: 0
arn001: 0
arn04: 0
arn09s18: 0
arn09s19: 0
arn09s

### Drop uninteresting attributes

In [33]:
dataDropped = data.copy()

dataDropped = dataDropped.drop(dataDropped[dataDropped.Label == "Unknown"].index)

attributesToDrop = ["Id" ,"Domain Name", "Flows", "Datetime", "Host Port", "Src IP Addr", "Dst IP Addr", "Client IP", "Host IP", "Src Port", "Dst Port"]
dataDropped = dataDropped.drop(attributesToDrop, axis=1)

In [34]:
printRow(dataDropped, id)

Row with id 1 not found in DataFrame.


## Convert Service label into integer

In [36]:
dataNumLabel = dataDropped.copy()

n = dataNumLabel["Label"].value_counts()

LABELS_DICT = {}
for i in range(len(n)):
    LABELS_DICT[n.index.tolist()[i]] = i
print()
print("LABELS AND CORRESPONDING NUMBER:")
for key,val in LABELS_DICT.items():
    print(f"{val}\t{key}")



LABELS AND CORRESPONDING NUMBER:
0	Youtube
1	Netflix
2	Browsing/Shopping
3	Twitch TV
4	Prime Video
5	SVT Play
6	Spotify
7	Facebook
8	Playstation
9	Soundcloud
10	Discord
11	Reddit
12	Amazon SHOP
13	Google Drive
14	Skype
15	Disney+
16	Steam Gaming
17	Gmail
18	Instagram
19	Outlook Mail
20	X


#### Conversion functions for labels

In [37]:
def label2num(label):
    return(LABELS_DICT[label])
def num2label(num):
    return next((key for key, val in LABELS_DICT.items() if val == num), None)

## Split the data from the labels

In [48]:
X = dataNumLabel.drop("Label", axis = 1).astype("float32").to_numpy()
Y = dataNumLabel["Label"].apply(label2num).to_numpy() #translate label to corresponding integer

ValueError: could not convert string to float: '1.0M'

### Trainging data and Testing data split & convert into tensor

In [42]:
# TRAIN TEST SPLIT
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=13)
 
# TO TENSOR
Xtrain = torch.tensor(Xtrain, dtype=torch.float32)
Ytrain = torch.tensor(Ytrain, dtype=torch.long)
Xtest = torch.tensor(Xtest, dtype=torch.float32)
Ytest = torch.tensor(Ytest, dtype=torch.long)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

## THE NETWORK

In [ ]:
# NETWORK CLASS
class Net(nn.Module):
    def __init__(self, input_size, h1, h2, h3, h4, output_size):
        super(Net, self).__init__()
        dropoutrate = 0.5
        self.fc1 = nn.Linear(input_size, h1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropoutrate)

        self.fc2 = nn.Linear(h1, h2)
        self.relu2 = nn.ReLU()

        self.fc3 = nn.Linear(h2, h3)
        self.relu3 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropoutrate)

        self.fc4 = nn.Linear(h3, h4)
        self.relu4 = nn.ReLU()

        self.fc5 = nn.Linear(h4, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout2(x)

        x = self.fc4(x)
        x = self.relu4(x)

        x = self.fc5(x)
        return x

### Create model

In [ ]:
# NETWORK LAYER SIZES
input_size = X.shape[1]
h1 = 200
h2 = 150
h3 = 100
h4 = 50
output_size = len(LABELS_DICT)

# CREATE NEURAL NETWORK MODEL
model = Net(input_size, h1, h2, h3, h4, output_size)

# LOSS & EVALUATION
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001) #create optimizer

## TRAINING

In [ ]:
# TRAINING
num_epochs = 500
for epoch in range(num_epochs):
    outputs = model(Xtrain) #Prediction
    loss = criterion(outputs, Ytrain) #Loss calculation

    optimizer.zero_grad() #reset optimizer gradient
    loss.backward() # bakåt ?
    optimizer.step() # uppdatera vikter

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

### Accuracy print

In [ ]:
# PRINT ACCURACY
with torch.no_grad():
    model.eval()
    outputs_test = model(Xtest)
    predicted_labels = torch.argmax(outputs_test, dim=1)
    acc = torch.sum(predicted_labels == Ytest).item() / len(Ytest)
    print(f"Test Accuracy: {acc}")

## Export Model !

In [ ]:
# export model weights here